# Peacock Creator

Use this notebook to create peacocks and visualize the results of breeding pairs, and to see the possible genetic makeup of various peacocks

The following code installs all neccessary code libraries for the visualization

In [17]:
%pip install -q ipywidgets


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [18]:
import ipywidgets as widgets

Use the following widget to add unique features to a peacock, and select the peacock's sex.  Any features that are not added to the peacock will be assumed to be wild type

In [19]:
# Define color genes
colorGenes = [
    # Note: the "default" gene aka Wild Type is not included because it is a special
    # case that can apply to alleles of any gene
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'o',       'name': 'Opal' },
    { 'notation': 'md',      'name': 'Midnight' },
    { 'notation': 'j',       'name': 'Jade' },
    { 'notation': 'mo',      'name': 'Montana' },
    { 'notation': 'ch',      'name': 'Charcoal' },
    { 'notation': 'st',      'name': 'Steel' },
    { 'notation': 'um',      'name': 'Ultramarine' },
    { 'notation': 'bu',      'name': 'Burnt Umber' }
]

sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' },
    { 'notation': 'Z(va)',   'name': 'Sonja\'s Violet' },
    { 'notation': 'Z(ve)',   'name': 'European Violet' },
    # Note: even though peach is actually 2 genes, we are treating
    # it as one for the sake of this code, and dealing with het Peach
    # as a special phenotype
    { 'notation': 'Z(pl:c)', 'name': 'Peach' }
]

# Define what genes combinations form special colors
multiGeneColors = [
    { 'name': 'Platinum',      'genes': ['br', 'o']}
]

sexAndAutosomalComboColors = [
    { 'name': 'Taupe',         'autosomalColor': 'o',  'sexColor': 'Z(pl)' },
    { 'name': 'Mocha',         'autosomalColor': 'md', 'sexColor': 'Z(pl)' },
    { 'name': 'Ivory',         'autosomalColor': 'o',  'sexColor': 'Z(c)' },
    { 'name': 'Indigo',        'autosomalColor': 'br', 'sexColor': 'Z(pl)' },
    { 'name': 'Hazel',         'autosomalColor': 'br', 'sexColor': 'Z(pl)' }
]

hetSexColors = [
    { 'name': 'Midway between Violet and Purple', 'alleles': ['Z(pl)', 'Z(ve)'] }
]

# Define pattern genes
patternGenes = [
    { 'notation': 'bs',      'name': 'Blackshoulder' }
]

# Define leucistic genes
leucisticGenes = [
    { 'notation': 'p',       'name': 'Pied' },
    { 'notation': 'WE',       'name': 'White Eye' }
]

piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]

whiteEyeAllotypes = [
    { 'notation': 'WE',      'name': 'White Eye' },
    { 'notation': 'sWE',     'name': 'Silver White Eye' }
]

# Define lucistic special cases
hetLeucistic = [
    { 'name': 'Pied', 'alleles': ['W', 'p'] }
]

In [23]:
# This function creates a handful of widgets that
# allow users to select the phenotype of a bird
# If we then assume all traits breed true, we
# can generate a genotype without making the
# user understand bird genetics
def createPeacockPhenotypeWidgets(peacockName):
    colorList = (['Wild Type'] + 
        # TECH EXPLANATION
        # lambda indicates an inline function
        # map() means that the lambda function is called for each item in the colorGenes array
        # It converts each gene's dictionary into just the name column
        # list() turns that map back into an array
        # The goal here is to make sure all genes show up as human readable options 
        # in the list, and they don't get out of sync with each other
        list(map(lambda gene: gene['name'], colorGenes)) + 
        list(map(lambda allotype: allotype['name'], sexLinkedColorAllotypes)) +
        list(map(lambda phenotype: phenotype['name'], multiGeneColors)) +
        list(map(lambda phenotype: phenotype['name'], sexAndAutosomalComboColors)) +
        list(map(lambda phenotype: phenotype['name'], hetSexColors)))
        
    
    color = widgets.Dropdown(
        options=colorList,
        value=colorList[0],
        description='Color:',
        disabled=False,
    )

    # See colorList for technical explanation
    patternList = (['Barred Wing Wild Type'] +
        list(map(lambda gene: gene['name'], patternGenes)))
    pattern = widgets.Dropdown(
        options=patternList,
        value=patternList[0],
        description='Pattern:',
        disabled=False,
    )
    
    # See colorList for technical explanation
    leucisticList = (['Non-Leucistic Wild Type'] +
       list(map(lambda allotype: allotype['name'], piedAllotypes)) +
       list(map(lambda allotype: allotype['name'], whiteEyeAllotypes)) +
       list(map(lambda phenotype: phenotype['name'], hetLeucistic)))
    
    leucistic = widgets.Dropdown(
        options=leucisticList,
        value=leucisticList[0],
        description='Leucistic:',
        disabled=False,
    )

    return (
        widgets.VBox([
            widgets.Label(value=peacockName),
            color, pattern, leucistic
        ]),
        {
            'color': color,
            'pattern': pattern,
            'leucistic': leucistic
        }
    )

display(widgets.Label(value="Enter phsyical traits for a breeding pair of peacocks. Note: All traits will be assumed to breed true"))
(maleWidgetsBox, maleWidgets) = createPeacockPhenotypeWidgets("Male Peacock")
(femaleWidgetsBox, femaleWidgets) = createPeacockPhenotypeWidgets("Female Peacock")
display(widgets.HBox([maleWidgetsBox,femaleWidgetsBox]))

Label(value='Enter phsyical traits for a breeding pair of peacocks. Note: All traits will be assumed to breed â¦

If you want to look at more complex pairings, including outcrosses, you can also edit the bird's geneotype below. Changes to the genotype will also change the phenotype inputs, so that both types of inputs show the same bird.

In [2]:
# This lets us turn a single gene into a genotype widget
# We will run this function over all the autosomal color genes
# and the shoulder genes to generate appr
def createGenotypeWidgetFromGene(gene):
    possibilities = [
        'WT/WT', 
        'WT/' + gene['notation'],
        gene['notation'] + '/' + gene['notation']
    ]
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description=gene['name'] + ':',
        disabled=False,
    )

def saveGenotypeToBird(bird, geneName, genotype):
    bird[geneName] = genotype

# This converts a phenotype to a genotype in a bird
# assuming the phenotype breeds true
def saveColorPhenotypeToBird(bird, phenotype, sex):
    foundPhenotype = False
    
    # Check normal color genes
    for gene in colorGenes:
        # If the current gene we are checking matches the selected
        # phenotype, set the genotype in the bird equal to homozygous
        # for the color
        if phenotype == gene['name']:
            bird[phenotype] = gene['notation'] + '/' + gene['notation']
            foundPhenotype = True
        # If the current gene we are checking does not match the phenotype
        # we should clear the genotype for that gene
        else:
            del bird[gene['name']]

    # Reset sex-linked colors to default wild type
    if sex == 'Male':
        bird['sex'] = 'Z(WT)/Z(WT)'
    else: 
        bird['sex'] = 'Z(WT)/w'
        
    # Check all sex-linked allotypes the same way we checked the autosomal
    # genes
    for allotype in sexLinkedColorAllotypes:
        if phenotype == allotype['name']:
            if sex == 'Male':
                bird['sex'] = allotype['notation'] + '/' + gene['notation']
            else: 
                bird['sex'] = allotype['notation'] + '/w'
            foundPhenotype = True

    # At this point, all color genes have been cleared
    # except for a matched phenotype
    # If we have found the matching phenotype already,
    # we can stop here
  

def getColorFromBird(bird):

SyntaxError: '[' was never closed (1940757381.py, line 27)

In [21]:
def createPeacockPhenotypeWidgets(peacockName):
    colorWidgets = list(map(lambda gene: 

SyntaxError: incomplete input (2756517185.py, line 2)